# CV Match Starter Notebook

This is a starter Notebook to download the dataset for your project and give some guidance in connecting to Nvidia GPU-powered models.

Make sure to copy or rename this notebook to avoid accidentally overwriting it later.

## Install Libraries

To install libraries you wish to use, type `!pip install <package>`

For instance, to install `pandas`, type `!pip install pandas`.

Below are some examples of libraries you may need, the code is commented out. Remove the `#` to execute. Uncomment one line at a time, to install the libraries one-by-one.

In [6]:
# !pip install langchain_nvidia_ai_endpoints
# !pip install googledriver
# !pip install openai
# !pip install pymupdf4llm

In [7]:
from googledriver import download_folder
from openai import OpenAI
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
from langchain_core.documents import Document
import pymupdf4llm

## Download the Data

The code below will download the relevant data for the CV Match project.

In [ ]:
# Google Drive location
URL = "https://drive.google.com/drive/folders/1-i4qvBR2M1f6IgxFAUuRLmvuH79t1xFT?usp=drive_link"

# Download Google Drive folder
download_folder(URL, 'cv_match')

## Extract Text from PDFs

The following code may be useful to you in extracting textual data from the provided PDF documents

In [10]:
# Extract text from sample CV
md_text = pymupdf4llm.to_markdown("cv_match/trainResumes/candidate_063.pdf")
print(md_text)

Processing cv_match/trainResumes/candidate_063.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
# Joseline Hernandez

## D A T A S C I E N T I S T A N D M A C H I N E L E A N R I N G E N G I N E E R


### Executive Profile

Good understanding of Data
Preprocessing, training and deployment of
machine learning and deep learning
models.. Worked on Time Series
Forecasting, Classification and Clustering
Problems. Hands on experience in NLP
tools.,

### Skills

Python, MySQL, Natural Language
Processing, Machine Learning, Neural
Networks, Flask, Docker, GIT, BERT, LSTM,
RNN, Deep Learning, AWS, RNN,
Tensorflow, Keras, PyTorch, Numpy,
Pandas, Glove, Lucene, Linux, Devops,
Spacy, GPU, Data Science.

### Other Activities

Data Analysis with python
Machine Learning with Python
Machine Learning Stanford University

### Projects

Study of factors affecting statistical
comparisons, such as the type of datas

## Modelling

### Option 1: Pre-Deployed Models

This section shows how to connect to models that have been deployed on prem for, available for you to use.

#### LLM

In [48]:
# Define LLM parameters for model LLama 3.1 8b
llm = ChatNVIDIA(
    base_url="http://nvpoc.ddnsfree.com:9901/v1",
    api_key="n/a",
    model="meta/llama-3.1-8b-instruct"
)

# Create generator object
events = llm.stream("Hi, write a short poem")

# Print elements of object
for e in events:
    print(e.content, end='')

Here is a short poem:

The sunset fades to night
A fiery sky, a gentle light
The stars appear, one by one
A diamond show, just begun

The world is hushed, a peaceful sight
The moon glows bright, with silvery light
The night air whispers, a soothing breeze
A calm descends, a peaceful ease

I hope you enjoy it! Is there a specific theme or subject you'd like me to write about?

#### Embedder

In [50]:
embeddings = NVIDIAEmbeddings(
    base_url="http://nvpoc.ddnsfree.com:9902/v1", 
    model="nvidia/nv-embedqa-e5-v5",
    truncate="END"
)

# This will return the vector embeddings for the word 'Hi'
# embeddings.embed_query("Hi")

#### Re-Ranker

In [2]:
reranker = NVIDIARerank(
    base_url="http://nvpoc.ddnsfree.com:9903/v1", 
    model="nvidia/nv-rerankqa-mistral-4b-v3",
    truncate="END"
)

In [29]:
# Example of using a re-ranker to rank passages by relevance to a query
query = "What is the GPU memory bandwidth of H100 SXM?"
passages = [
    "The Hopper GPU is paired with the Grace CPU using NVIDIA's ultra-fast chip-to-chip interconnect, delivering 900GB/s of bandwidth, 7X faster than PCIe Gen5. This innovative design will deliver up to 30X higher aggregate system memory bandwidth to the GPU compared to today's fastest servers and up to 10X higher performance for applications running terabytes of data.", 
    "A100 provides up to 20X higher performance over the prior generation and can be partitioned into seven GPU instances to dynamically adjust to shifting demands. The A100 80GB debuts the world's fastest memory bandwidth at over 2 terabytes per second (TB/s) to run the largest models and datasets.", 
    "Accelerated servers with H100 deliver the compute power—along with 3 terabytes per second (TB/s) of memory bandwidth per GPU and scalability with NVLink and NVSwitch™.", 
]

# Define Re-ranker
response = reranker.compress_documents(
  query=query,
  documents=[Document(page_content=passage) for passage in passages]
)

# Print Ranked Responses
for x in response:
    print("Score: ", x.metadata.get('relevance_score'))
    print("Passage: ", x.page_content, '\n')

Score:  8.8359375
Passage:  Accelerated servers with H100 deliver the compute power—along with 3 terabytes per second (TB/s) of memory bandwidth per GPU and scalability with NVLink and NVSwitch™. 

Score:  0.29736328125
Passage:  The Hopper GPU is paired with the Grace CPU using NVIDIA's ultra-fast chip-to-chip interconnect, delivering 900GB/s of bandwidth, 7X faster than PCIe Gen5. This innovative design will deliver up to 30X higher aggregate system memory bandwidth to the GPU compared to today's fastest servers and up to 10X higher performance for applications running terabytes of data. 

Score:  -0.10614013671875
Passage:  A100 provides up to 20X higher performance over the prior generation and can be partitioned into seven GPU instances to dynamically adjust to shifting demands. The A100 80GB debuts the world's fastest memory bandwidth at over 2 terabytes per second (TB/s) to run the largest models and datasets. 



### Option 2: API Endpoint

If you would like to use other Nvidia models, feel free to browse their catalog 
https://build.nvidia.com/explore/discover

You will first need to create a Developer account with them. If you use your ADCB email address, you will be eligible for 5000 free credits. Otherwise, you will be given 1000 free credits. These will be consumed as you query the API.

From the catalog, you can insert Python the code snippet (sample shown below). Just make sure to replace `my_api_key` with your own and `model` with the one you wish to query.

In [34]:
# Generate API key from the Nvidia catalog site
my_api_key = 'INSERT YOUR API KEY HERE'

In [39]:
# Define LLM paramters
client = ChatNVIDIA(
  model="meta/llama-3.2-3b-instruct",
  api_key=my_api_key, 
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
)

# Print out reponse to a query
for chunk in client.stream([{"role":"user",
                             "content":"Write a limerick about the wonders of GPU computing."}]): 
  print(chunk.content, end="")


Here is a limerick about GPU computing:

There once was a GPU so fine,
Whose computing power was truly divine.
It processed with speed,
And calculations with ease,
And made complex tasks truly sublime.

## Test Data for Scoring your Model

For this project, the test data is available in the folder `testResumes`.